In [2]:
import os
import openai
import pandas as pd
import pdfplumber
import json
import numpy as np

import config

In [3]:
from openpyxl import load_workbook

file_path = './truth_excel/test_all_studies_final.xlsx'
workbook = load_workbook(file_path)
sheet = workbook.active

# Convert to DataFrame
data = sheet.values
columns = next(data)
df_truth = pd.DataFrame(data, columns=columns)

In [4]:
import openai
from openai import OpenAI
client = OpenAI()

In [5]:
def get_file_names(directory):
    """Gets a list of file names in the specified directory.

    Args:
        directory (str): The path to the directory.

    Returns:
        list: A list of file names.
    """

    file_names = []
    for entry in os.scandir(directory):
        if entry.is_file():
            file_names.append(entry.name)
    return file_names

directory_path = "../Desktop/Testing"
file_names = get_file_names(directory_path)
print(file_names)

['rynn2008.pdf', 'lennox2003.pdf', 'kasper2014.pdf', 'hartford2007.pdf', 'boyer2004.pdf', 'merideth2012.pdf', 'mahablesh2013.pdf', 'davidson2004.pdf', 'pollock2001.pdf', 'nicolini2009.pdf', 'allgulander2004.pdf', 'pollock2008a.pdf', 'wu2011.pdf', 'bose2008.pdf', 'rickels2003.pdf', 'rothschild2012.pdf', 'stein2008.pdf', 'khan2011.pdf', 'alaka2014.pdf', 'ball2015.pdf', 'nimatoudis2004.pdf']


# Pollock2008a was omitted due to heavy data discrepencies 
# Don't know if I'll include Khan...it's all NA's. also khan suggests to omit washout periods...Khan is all NA's as it uses link to supplemental figures

In [ ]:
# Function to extract text and tables
def extract_pdf_text(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            # Extract text
            text += page.extract_text() + "\n"
    return text


def query_gpt4_text(text):
    structured_prompt = (
        f"Clinical Trial Report Analysis:\n\n"
        f"Extracted Text:\n{text}\n\n"
        f"This is a clinical trial report. For EACH intervention in the trial (including placebo), please extract the following characteristics and format the response as valid JSON using this exact example structure:\n\n"
        f"Example format:\n"
        f"{{\n"
        f'    "Last Name of Main Author and Year": "Doe et al., 2021",\n'
        f'    "Full Population Sample Size": "451",\n'
        f'    "Intervention": "Duloxetine: 50 mg/day",\n'
        f'    "Main Race": "White",\n'
        f'    "Percent of Intervention Population that is Female (%)": "61.5",\n'
        f'    "Mean HAMA Score": "24.5",\n'
        f'    "Mean Population Age (Year)": "43.2",\n'
        f'    "Attrition Rate (%)": "30.2",\n'
        f'    "Full Sponsor Name": "ABC Pharmaceuticals",\n'
        f'    "Follow-up Time (Weeks)": "10",\n'
        f'    "Diagnostic Criteria": "DSM-IV"\n'
        f"}}\n\n"
        f"'Full Population Sample Size' should refer to the TOTAL population enrolled in the study, across all interventions and groups, not just the population size for the specific intervention.\n"
        f"'Intervention' should be in mg/day, not any other unit of measurement.\n"
        f"'Intervention' should be the total range of the dose or whateverrreerer.\n"
        f"'Intervention' should remove any specific formatation names.\n"
        f"'follow up time should refer to total length of the study.\n"
        f"'sample size' should be the total amount that were randomly assigned to an intervention\n"
        f"'Mean HAMA' should be the mean hama at the beginning of the study\n"
        f"'attrition rate should be number of patients who completed the treatment period after assignment\n"
        f"Make sure each JSON object follows this format exactly. For any missing or unavailable data, input 'NA'."
    )
    response = completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a data extraction assistant extracting data from clinical trial reports."},
                  {"role": "user", "content": structured_prompt}],
    )
    return response